In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("KDDCup_PySpark").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(sc)

22/11/05 23:23:55 WARN Utils: Your hostname, pc resolves to a loopback address: 127.0.1.1; using 192.168.170.52 instead (on interface wlp3s0)
22/11/05 23:23:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/05 23:23:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[*] appName=KDDCup_PySpark>


In [2]:
rdd = sc.textFile("./data/kddcup.data.gz")

In [3]:
rdd.getNumPartitions()

rdd = rdd.repartition(10)
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

#rdd.persist()

8
10


In [4]:
# Get ten record randomly
rdd.takeSample(False, 10, 42)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,258,16,1.00,1.00,0.00,0.00,0.06,0.06,0.00,255,16,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,112,9,1.00,1.00,0.00,0.00,0.08,0.06,0.00,255,9,0.04,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,202,2,1.00,1.00,0.00,0.00,0.01,0.05,0.00,255,2,0.01,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,246,20,1.00,1

In [5]:
# count elements
total_count = rdd.count()
print(total_count)

4898431


In [6]:
# calculate the ratio of normal connections
normal_count = rdd.filter(lambda x: x.split(",")[-1] == "normal.").count()
print(f"Ration of normal connection is {round(normal_count/total_count,4)*100} %")

Ration of normal connection is 19.86 %


In [7]:
# Get the list of labels
label_rdd = rdd.map(lambda x: x.split(",")[-1]).distinct()
label_rdd.collect()

['neptune.',
 'loadmodule.',
 'warezclient.',
 'pod.',
 'smurf.',
 'nmap.',
 'spy.',
 'back.',
 'teardrop.',
 'ipsweep.',
 'multihop.',
 'phf.',
 'ftp_write.',
 'guess_passwd.',
 'normal.',
 'land.',
 'satan.',
 'imap.',
 'portsweep.',
 'warezmaster.',
 'rootkit.',
 'buffer_overflow.',
 'perl.']

In [8]:
# count the number of connection for each label
rdd.map(lambda x: (x.split(",")[-1],1)).reduceByKey(lambda x,y: x+y).collect()

[('neptune.', 1072017),
 ('loadmodule.', 9),
 ('warezclient.', 1020),
 ('pod.', 264),
 ('smurf.', 2807886),
 ('nmap.', 2316),
 ('spy.', 2),
 ('back.', 2203),
 ('teardrop.', 979),
 ('ipsweep.', 12481),
 ('multihop.', 7),
 ('phf.', 4),
 ('ftp_write.', 8),
 ('guess_passwd.', 53),
 ('normal.', 972781),
 ('land.', 21),
 ('satan.', 15892),
 ('imap.', 12),
 ('portsweep.', 10413),
 ('warezmaster.', 20),
 ('rootkit.', 10),
 ('buffer_overflow.', 30),
 ('perl.', 3)]

In [9]:
# Get the connection type with successful root_shell connections to servers,
# where the number of data bytes from source(src_bytes) is 500 times more than from server (dst_types)

splited_rdd = rdd.map(lambda x: x.split(","))

splited_rdd.filter(lambda x: x[13] == "1") \
            .map(lambda x: (x[1], x[4], x[5])) \
            .filter(lambda x: int(x[1]) < 500 * int(x[2])) \
            .map(lambda x: x[0]).distinct().collect()

['tcp']

In [10]:
# Get the list of Protocols that are normal and vulnerable to attacks, 
# where there is NOT guest login to the destination

splited_rdd.filter(lambda x: x[-1] == 'normal.' and x[21] == "0") \
            .map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).collect()

[('icmp', 12763), ('udp', 191348), ('tcp', 764894)]

In [11]:
# get a summary statstics for the sum of the tcp connections to the same 
# destination IP address(hint : protocol_type and dst_host_count features)
from pyspark.mllib.stat import Statistics
from math import sqrt

summary = Statistics.colStats(splited_rdd.filter(lambda x: x[1] == "tcp").map(lambda x: [int(x[31])]))

print("mean :", summary.mean())
print("min :", summary.min())
print("max :", summary.max())
print("standard deviation :", sqrt(summary.variance()))

mean : [201.7520146]
min : [0.]
max : [255.]
standard deviation : 90.72575504478223


In [12]:
# filter the number of icmp-based attacks for each service
splited_rdd.filter(lambda x: x[1] == 'icmp' and x[-1] != 'normal.') \
            .map(lambda x: (x[2],1)) \
            .reduceByKey(lambda x,y: x+y) \
            .collect()

[('eco_i', 12570), ('urp_i', 3), ('ecr_i', 2808204), ('tim_i', 5)]